In [ ]:
# Explorating the different API requests
from urllib3 import request
import os
import json
import pandas as pd
import io
from dotenv import load_dotenv
import json
from pathlib import Path
import numpy as np 
from shapely.geometry import shape, mapping
from shapely.ops import orient
_ = load_dotenv()

In [ ]:
# Getting the API key
API_KEY = os.getenv("STATS_NZ_API_KEY_PRIMARY")

In [ ]:
# Below is the link to view the data source as a table:
# https://explore.data.stats.govt.nz/vis?pg=0&snb=1&df[ds]=ds-nsiws-disseminate&df[id]=CEN23_TBT_008&df[ag]=STATSNZ&df[vs]=1.0&isAvailabilityDisabled=false&dq=raTotal%2Bra00%2Bra02%2BegTS%2Beg6%2Beg5%2Beg4%2Beg3%2Beg2%2Beg1%2Brb3%2BrbTS%2Brb2%2Brb1.05.2023&to[TIME]=false&hc[Variable%20codes]=&tm=rainbow%20sa3

STATS_URL = "https://api.data.stats.govt.nz/rest/data/STATSNZ,CEN23_TBT_008,1.0/ra05+ra04+raTotal+ra00+ra02+egTS+eg6+eg5+eg4+eg3+eg2+eg1+rb3+rbTS+rb2+rb1..2023?dimensionAtObservation=AllDimensions&format=csvfilewithlabels"

response = request(
    method="GET"
    , url = STATS_URL
    , headers={
        "Ocp-Apim-Subscription-Key":API_KEY

    }
)

data = response.data.decode("utf-8")

df = pd.read_csv(io.StringIO(data))

In [ ]:
# View the first few rows
df.head(5)

In [ ]:
SOURCE_COLS = {
    "Area": "area_name"
    ,  "CEN23_TBT_GEO_006": "area_code"
    , "Variable codes":"feature"
    , "OBS_VALUE": "count"
}

RENAME = {
    # Ethnicity
    "Middle Eastern/Latin American/African":"mena"
    , "Other ethnicity":"other_ethnicity"
    , "Pacific Peoples": "pasifika"
    , "Asian": "asian"
    , "European": "nz_european"
    , "Māori": "māori"
    , "Total stated - ethnicity": "total_ethnicity"
    # LGBT
    , "LGBTIQ+":"lgbt"
    , "Not LGBTIQ+":"non_lgbt"
    # Religion - THIS NEEDS TO BE FIXED AT A LATER DATE SINCE RELIGION MAY NOT ADD UP TO 100%
    , "No religion":"no_religion"
    , "Christian":"christian"
    , "Islam": "islam"
    , "Judaism" : "judaism"
    , "Religious affiliation (total responses) - total census usually resident population count": "total_religion"
}


pivoted_df = df[SOURCE_COLS.keys()]\
    .rename(columns=SOURCE_COLS)\
    .pivot(index=["area_name", "area_code"], columns="feature", values="count")\
    .reset_index()\
    .rename(columns=RENAME)\
    [["area_name", "area_code"] + list(RENAME.values())]\
    .assign(
        total_lgbt=lambda x: (x["non_lgbt"] + x["lgbt"])
        , other_religion=lambda x: (x["total_religion"] - (x["christian"] + x["islam"] + x["judaism"] + x["no_religion"]) )
    )\
    .query("total_ethnicity.notna()")\
    .fillna(0)



# After looking at the few records with N/A I decided to replace with 0

In [ ]:
pivoted_df.head(5)

In [ ]:
# NA Test (Should be 0)
total_rows = len(pivoted_df)
for col in pivoted_df.columns:
    na_count = len(pivoted_df.query(f"{col}.isna()"))
    if na_count == 0:
        pass
    else:
        print(f"COLUMN {col}: missing {na_count}")

In [ ]:
AREAS = {
    "region":("./data/area_source/regional-council-2025-clipped.json", "REGC2025_V1_00")
    , "territorial":("./data/area_source/territorial-authority-2025-clipped.json", "TA2025_V1_00")
    , "urban_area": ("./data/area_source/urban-rural-2025-clipped.json", "UR2025_V1_00")
    , "sa3": ("./data/area_source/statistical-area-3-2025-clipped.json", "SA32025_V1_00")
    , "sa2": ("./data/area_source/statistical-area-2-2025-clipped.json", "SA22025_V1_00")
}
# json.loads(Path(link).read_text("utf-8"))
loaded_maps = [
    (json.loads(Path((AREAS[x][0])).read_text("utf-8")),AREAS[x][1], x) for x in AREAS
]


In [ ]:
no_data = 0 # Track areas with no data
duplicates = 0 # Track any duplicates
rows_added = 0 # keep count of what rows have been joined with a feature
error_cols = 0

ETH_COLS = ["mena", "other_ethnicity", "pasifika", "asian", "nz_european", "māori"]
ETH_SUM = "total_ethnicity"

LGBT_COLS = ["lgbt", "non_lgbt"]
LGBT_SUM = "total_lgbt"

RELI_COLS = ["christian", "islam", "judaism", "no_religion"] # Excluded other religion for now
RELI_SUM = "total_religion"

max_percent_by_region = dict()


for loaded_map, area_col, map_name in loaded_maps:

    # Set the max percentages for each area level
    max_percent_by_region[map_name] = dict({x:[] for x in ETH_COLS + LGBT_COLS + RELI_COLS})

    for area in loaded_map["features"]:


        geom = shape(area['geometry'])
        fixed_geom = orient(geom, sign=-1.0) 
        area['geometry'] = mapping(fixed_geom)

        # Get the name and area code
        area_code =  area["properties"][area_col]
        area_name = area["properties"][area_col + "_NAME"]

        region_ethnicity_data = pivoted_df.query("area_code == @area_code")

        if len(region_ethnicity_data) == 0:
            no_data+=1
        elif len(region_ethnicity_data) == 1:


            as_records = region_ethnicity_data.to_dict(orient='records')[0]

            demographics_obj = {
                "area_name": as_records["area_name"]
                , "area_code": as_records["area_code"]
            }

            try:

                for eth_col in ETH_COLS:
                    # Maybe add rounding??
                    
                    if not demographics_obj.get(eth_col):
                        demographics_obj[eth_col]=dict()

                    # Don't contribute  where there is less than 20 people in an area
                    if as_records[ETH_SUM] > 20: 
                        max_percent_by_region[map_name][eth_col].append(as_records[eth_col] / as_records[ETH_SUM])

                    demographics_obj[eth_col]["pct"] = as_records[eth_col] / as_records[ETH_SUM]
                    demographics_obj[eth_col]["count"] = as_records[eth_col]


                for lgbt_col in LGBT_COLS:
                    # Maybe add rounding??
                    if not demographics_obj.get(lgbt_col):
                        demographics_obj[lgbt_col]=dict()

                    # Don't contribute  where there is less than 20 people in an area
                    if as_records[LGBT_SUM] > 20: 
                        max_percent_by_region[map_name][lgbt_col].append(as_records[lgbt_col] / as_records[LGBT_SUM])
                    
                    demographics_obj[lgbt_col]["pct"] = as_records[lgbt_col]  / as_records[LGBT_SUM]
                    demographics_obj[lgbt_col]["count"] = as_records[lgbt_col]

                for reli_col in RELI_COLS:
                    # Maybe add rounding??
                    if not demographics_obj.get(reli_col):
                        demographics_obj[reli_col]=dict()

                    # Don't contribute  where there is less than 20 people in an area
                    if as_records[RELI_SUM] > 20: 
                        max_percent_by_region[map_name][reli_col].append(as_records[reli_col] / as_records[RELI_SUM])

                    demographics_obj[reli_col]["pct"] = as_records[reli_col]  / as_records[RELI_SUM]
                    demographics_obj[reli_col]["count"] = as_records[reli_col]

                demographics_obj["total"] = as_records[ETH_SUM]


                area["properties"]["demographics"]=demographics_obj

            except ZeroDivisionError as e:
                error_cols += 1

            

            rows_added+=1
        elif len(region_ethnicity_data) > 1:
            duplicates+=1

print(f"{no_data} areas with no data from census")
print(f"{duplicates} areas with duplicate data from census")
print(f"{rows_added} areas with data added from census")
print(f"{error_cols} areas with errors")


print(max_percent_by_region)


In [ ]:
percentile95_percent_by_region = dict()

for area in max_percent_by_region:
    percentile95_percent_by_region[area] = dict()
    for col in max_percent_by_region[area]:
 

        # For 

        percentile = np.percentile(max_percent_by_region[area][col], 99.9)
        _max = max(max_percent_by_region[area][col])

        if area in ["region", "territorial"]: # use the max
            percentile95_percent_by_region[area][col] = _max
        else: # For SA2 & SA3 use percentile 90+9.9 to exclude the extreme outliers
            percentile95_percent_by_region[area][col]= percentile

        



In [ ]:
TARGET_PATH = "./data/area_ethnicity/"

for loaded_map, _, name in loaded_maps:
    path = Path(TARGET_PATH + name + ".json")


    with open(path, "w", encoding="utf-8") as f:
        json.dump(loaded_map,f)



max_path = Path(TARGET_PATH + "max_pct_by_area.json")

with open(max_path, "w", encoding="utf-8") as f:
    json.dump(percentile95_percent_by_region,f)